![manufacturing gears](manufacturing.jpg)

Manufacturing processes for any product is like putting together a puzzle. Products are pieced together step by step, and keeping a close eye on the process is important.

For this project, you're supporting a team that wants to improve how they monitor and control a manufacturing process. The goal is to implement a more methodical approach known as statistical process control (SPC). SPC is an established strategy that uses data to determine whether the process works well. Processes are only adjusted if measurements fall outside of an acceptable range. 

This acceptable range is defined by an upper control limit (UCL) and a lower control limit (LCL), the formulas for which are:

$ucl = avg\_height + 3 * \frac{stddev\_height}{\sqrt{5}}$

$lcl = avg\_height - 3 * \frac{stddev\_height}{\sqrt{5}}$

The UCL defines the highest acceptable height for the parts, while the LCL defines the lowest acceptable height for the parts. Ideally, parts should fall between the two limits.

Using SQL window functions and nested queries, you'll analyze historical manufacturing data to define this acceptable range and identify any points in the process that fall outside of the range and therefore require adjustments. This will ensure a smooth running manufacturing process consistently making high-quality products.

## The data
The data is available in the `manufacturing_parts` table which has the following fields:
- `item_no`: the item number
- `length`: the length of the item made
- `width`: the width of the item made
- `height`: the height of the item made
- `operator`: the operating machine

In [8]:
WITH HeightStats AS (
    SELECT
        operator,
        item_no,
        ROW_NUMBER() OVER (PARTITION BY operator ORDER BY item_no) AS row_number,
        height,
        AVG(height) OVER (PARTITION BY operator ORDER BY item_no ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS avg_height,
        STDDEV(height) OVER (PARTITION BY operator ORDER BY item_no ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS stddev_height
    FROM
        manufacturing_parts
),
ControlLimits AS (
    SELECT
        operator,
        row_number,
        item_no,
        height,
        avg_height,
        stddev_height,
        (avg_height + 3 * stddev_height) AS ucl,
        (avg_height - 3 * stddev_height) AS lcl
    FROM
        HeightStats
    WHERE
        row_number >= 5
),
Alerts AS (
    SELECT
        operator,
        row_number,
        height,
        avg_height,
        stddev_height,
        ucl,
        lcl,
        CASE
            WHEN height <= ucl AND height >= lcl THEN TRUE
            ELSE FALSE
        END AS alert
    FROM
        ControlLimits
    ORDER BY
        item_no
)
SELECT
    operator,
    row_number,
    height,
    avg_height,
    stddev_height,
    ucl,
    lcl,
    alert
FROM
    Alerts
LIMIT 57;

,operator,row_number,height,avg_height,stddev_height,ucl,lcl,alert
0,Op-1,5,19.46,19.778,1.062812,22.966437,16.589563,True
1,Op-1,6,20.36,19.912,1.090812,23.184435,16.639565,True
2,Op-1,7,20.22,20.030,1.084574,23.283721,16.776279,True
3,Op-1,8,21.03,19.934,0.931225,22.727675,17.140325,True
4,Op-1,9,19.78,20.170,0.598832,21.966497,18.373503,True
5,Op-1,10,20.71,20.420,0.476812,21.850437,18.989563,True
6,Op-1,11,20.62,20.472,0.482773,21.920320,19.023680,True
7,Op-1,12,19.51,20.330,0.650654,22.281961,18.378039,True
8,Op-1,13,20.06,20.136,0.521565,21.700695,18.571305,True
9,Op-1,14,20.30,20.240,0.483270,21.689810,18.790190,True
